# Présentation

Ce notebook contient les consignes ainsi que des emplacements vides pour apporter les réponses aux questions posées.

Ne changez pas les noms de variable qui vous sont proposées.




# Echauffement

## Création DataFrame à partir de 0 

A partir d'une base de données SQLite3 dont le code python est présent ci-dessous, nous allons recréer un dataframe manuellement basée sur les mêmes données. En seconde partie, nous verrons comment convertir un résultat de requêtes SQL directement en dataframe

```csv
fruit,origin,quantity
apple,france,12
apple,germany,78
apple,france,19
orange,france,10
orange,germany,98
strawberry,spain,2
```

la requête SQL pour créer la table contenant ces données

```sql
CREATE TABLE basket(fruit text, origin text, quantity int)
```


In [ ]:
import pandas as pd

Créer une série représentant la colonne `fruit` en utilisant le constructeur [`pd.Series`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html)

In [ ]:
fruit = pd.Series(['apple', 'apple', 'apple', 'orange', 'orange', 'strawberry'])

Créer une série représentant la colonne `origin` en utilisant le constructeur [`pd.Series`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html)

In [ ]:
origin = pd.Series(['france', 'germany', 'france', 'france', 'germany', 'spain'])

Créer une série représentant la colonne `quantity` en utilisant le constructeur [`pd.Series`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html)

In [ ]:
quantity = pd.Series([12, 78, 19, 10, 98, 2])

Créer un dataframe composé de ces 3 séries créées précédemment grâce au constructeur [`pd.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html#pandas.DataFrame)

In [ ]:
basket = pd.DataFrame({'fruit': fruit, 'origin': origin, 'quantity': quantity})
basket.info()

## Création DataFrame à partir d'un fichier CSV

Vous obtenez une seconde série de données, via un fichier CSV cette fois-ci. Vous pouvez donc lire le fichier CSV directement via la méthode `read_csv`.

Ce fichier CSV contient les informations suivantes

```csv
fruit,origin,quantity
kiwi,uk,12
kiwi,new zealand,78
pineapple,france,19
blueberry,canada,10
blueberry,canada,98
strawberry,spain,2
```

In [ ]:
import io

data = ''
data += 'fruit,origin,quantity\n'
data += 'kiwi,uk,12\n'
data += 'kiwi,new zealand,78\n'
data += 'pineapple,france,19\n'
data += 'blueberry,canada,10\n'
data += 'blueberry,canada,98\n'
data += 'strawberry,spain,2'

output = io.StringIO(data)


basketTom = pd.read_csv(output)
basketTom.info()

Combiner les dataframes basket et basketTom en un nouveau dataframe basketFull

In [ ]:
basketFull = pd.concat([basket, basketTom])
basketFull.info()

## Requêtes sur le DataFrame
Pour répondre à chacune des questions, proposer une requête sur le dataframe `basketFull`. Attention à ne jamais modifier le dataframe `basketFull`.

Quelle est la quantité max par pays d'origine tous fruits confondus ?

In [ ]:
basketFull[['origin', 'quantity']].groupby('origin').max()

Quelle est la quantité max par fruit et par pays d'origine ?

In [ ]:
basketFull.groupby(['fruit', 'origin']).max()

Quelle est la quantité totale par fruit ?

In [ ]:
basketFull[['fruit', 'quantity']].groupby('fruit').sum()

Afficher dans un graphique de barres empilées la quantité de fruits transférés.

In [ ]:
basketFull.groupby(['fruit', 'origin']).max()['quantity'].info()

In [ ]:
basketFull.groupby(['fruit', 'origin']).sum()['quantity'].unstack('origin').plot.bar(stacked=True, rot = 45)

# Transactions immobilières

Pour cette seconde partie, nous repartons sur la collecte des transactions immobilières et nous allons tenter d'identifier les caractéristiques les plus pertinentes.

Télécharger le [fichier des transactions immobilières](https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20230405-160733/valeursfoncieres-2022.txt)

Créer un premier dataframe qui ne va conserver que les transactions relatives à Charleville-Mézières.

Modifier le dataframe pour convertir les montants en k€.

Conserver dans le dataframe uniquement les colonnes utiles :
- No voie
- Type de voie
- Code voie
- Voie
- Code postal
- Commune
- Code departement
- Code commune
- Valeur fonciere
- Nombre pieces principales
- Surface Carrez du 1er lot

Grâce l'[API Adresse](https://adresse.data.gouv.fr/api-doc/adresse), envoyer votre fichier d'adresse pour récupérer les coordonnées GPS correspondantes

Construire un nouveau dataframe contenant uniquement latitude, longitude, valeur foncière, surface et nbre de pièces


Afficher un 1er graphique x = latitude, y = longitude et la couleur correspondant au prix sous forme de scatter.

Indiquer quelle est la valeur de la transaction immobilière la plus importante.

Où se situe le centre géographique de toutes les transactions enregistrées ?


In [ ]:
import pandas as pd
import io

In [ ]:
immobilier = pd.read_csv('./valeursfoncieres-2022.txt', sep='|')
immobilier.info()

In [ ]:
immobilier['Code postal'].head()

In [ ]:
charleville = immobilier[immobilier['Commune'].str.match('charleville.+', case=False)]

In [ ]:
charleville['Commune'].unique()

In [ ]:
charleville = charleville[['No voie', 'Type de voie', 'Code voie', 'Voie', 'Code postal', 'Commune', 'Code departement', 'Code commune', 'Valeur fonciere', 'Nombre pieces principales', 'Surface Carrez du 1er lot']]

In [ ]:
charleville['No voie'] = charleville['No voie'].fillna(0)

In [ ]:
charleville['Code postal']

In [ ]:
def convert_to_str(x):
    return str(int(x))

search = pd.DataFrame()
search['adress'] = charleville['No voie'].apply(convert_to_str) + ', ' + charleville['Voie']
search['postcode'] = charleville['Code postal']

search['city'] = charleville['Commune']

In [ ]:
dataCSV = search.to_csv(None) #convert to string

In [ ]:
print(dataCSV)

In [ ]:
import requests

In [ ]:
# multipart file
# -F column=adress, -F column=posstcode
# -F result_columns=latitude
# -F result_columnss=longitude


files = [
    ('data', dataCSV),
    ('columns', (None, 'adress')),
    ('columns', (None, 'postcode')),
    ('result_columns', (None, 'latitude')),
    ('result_columns', (None, 'longitude'))
]

response = requests.post('https://api-adresse.data.gouv.fr/search/csv/', files=files)



In [ ]:
charlevilleLonLat = pd.read_csv(io.StringIO(response.text))

In [ ]:
charlevilleLonLat.columns

In [ ]:
charlevilleLonLat.info()

In [ ]:
charleville.info()

In [ ]:
#Join with key unnamed both dataframe
charlevilleFull = charleville.join(charlevilleLonLat.set_index('Unnamed: 0'), lsuffix='_origin', rsuffix='_api')

In [ ]:
charlevilleFull.info()

In [ ]:
charlevilleFull['Valeur fonciere'] = charlevilleFull['Valeur fonciere'].str.replace(',', '.', regex=True).astype('float32')

In [ ]:
charlevilleFull.plot.scatter(x='longitude', y='latitude', c='Valeur fonciere')

In [ ]:
charlevilleFull.to_csv('./charleville-full.csv')

In [ ]:
charlevilleLonLat.to_csv('./charleville-lite.csv')

In [ ]:
charleville.to_csv('./charleville.csv')

In [ ]:
immobilier.to_csv('./immobilier.csv')